<a href="https://colab.research.google.com/github/sswwdk/wn7_dl_rep/blob/main/1_%EC%8B%A0%EA%B2%BD%EB%A7%9D%EC%9D%98_%EA%B5%AC%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.__version__

'2.19.0'

In [ ]:
#iris dataset load
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# MLPClassifier 로 신경망 구성

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,30), activation="logistic", solver="adam",
                    max_iter=1000)
mlp.fit(X_train, y_train)
mlp.score(X_test, y_test)
pred = mlp.predict(X_test)

import pandas as pd
import numpy as np
pd.crosstab(pred, y_test) #교차분류표

col_0,0,1,2
row_0,,,
0,14,0,0
1,0,18,0
2,0,0,13


# 텐서플로로 신경망 구성

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
X = tf.placeholder(tf.float32, [None, 4])
y = tf.placeholder(tf.float32, [None, 3])


## W 가중치, b 편향 - 1layer


In [ ]:
W1 = tf.Variable(tf.random_normal([4, 50]))
b1 = tf.Variable(tf.zeros([50]))
h1 = tf.nn.sigmoid(tf.matmul(X, W1)+b1)

## 2layer - 30 node 로 구성

In [ ]:
W2 = tf.Variable(tf.random_normal([50, 30]))
b2 = tf.Variable(tf.zeros([30]))
h2 = tf.nn.sigmoid(tf.matmul(h1, W2)+b2)

## 3layer - 출력층

In [ ]:
W3 = tf.Variable(tf.random_normal([30, 3]))
b3 = tf.Variable(tf.zeros([3]))
h3 = tf.nn.softmax(tf.matmul(h2, W3)+b3)

In [ ]:
cross_entropy = -tf.reduce_sum(y*tf.log(h3), reduction_indices=[1])
loss = tf.reduce_mean(cross_entropy)
train = tf.train.AdamOptimizer().minimize(loss)

In [ ]:
import pandas as pd

feed_train = {X: X_train, y: pd.get_dummies(y_train).values}

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(1000):
    sess.run(train, feed_dict=feed_train)

    if i%100 == 0:
        tr_loss = sess.run(loss, feed_dict=feed_train)
        print("Step %d, %.5f" % (i, tr_loss))

Step 0, 1.77492
Step 100, 0.28717
Step 200, 0.14428
Step 300, 0.10442
Step 400, 0.08143
Step 500, 0.06789
Step 600, 0.06117
Step 700, 0.05704
Step 800, 0.05370
Step 900, 0.05101


In [ ]:
correct_prediction = tf.equal(tf.argmax(h3, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
feed_test = {X: X_test, y: pd.get_dummies(y_test).values}
print(sess.run(accuracy, feed_dict=feed_test))

1.0


In [ ]:
sess.close()

# keras로 신경망 구성

In [ ]:
#iris dataset load
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
from tensorflow.keras.models import Sequential
model = Sequential()
model

<Sequential name=sequential_1, built=False>

In [ ]:
from tensorflow.keras.layers import Input, Dense
model.add(Input(shape=(4,)))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(3, activation='softmax'))


In [ ]:
model

<Sequential name=sequential_1, built=True>

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(X_train, y_train, epochs=300, verbose=1)

Epoch 1/300


RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.

In [ ]:
model.evaluate(X_test, y_test)

# 파이토치로 구성하기

In [ ]:
import torch
torch.__version__

'2.8.0+cu126'

In [ ]:
if torch.cuda.is_available():
  torch.cuda.get_device_name(0)

In [ ]:
#iris dataset load
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
import torch

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [ ]:
import torch.nn as nn
model = nn.Sequential(
    nn.Linear(4,50), nn.Sigmoid(),
    nn.Linear(50,30), nn.Sigmoid(),
    nn.Linear(30,3), nn.Softmax(dim=1),
)

In [ ]:
import torch.optim as optim

#w의 초기값 설정
for layer in model:
  if isinstance(layer, nn.Linear):
    nn.init.xavier_uniform_(layer.weight)

#손실함수
crit = nn.CrossEntropyLoss()
#옵티마이저
opt = optim.Adam(model.parameters())

In [ ]:
#학습설정
for epoch in range(300):
  outputs = model(X_train)
  loss = crit(outputs, y_train.long()) #손실값
  opt.zero_grad() #새로운 w
  loss.backward()
  opt.step()


In [ ]:
# 모델평가
with torch.no_grad():
  test_outputs = model(X_test)
  _, predicted = torch.max(test_outputs, 1)
  accuracy = (predicted==y_test).sum().item()/y_test.size(0)
  print(loss.item(), accuracy)

0.6326429843902588 0.9777777777777777


In [ ]:
# keras 함수형api 로 신경망 구성하기

In [1]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import concatenate, Activation

input = Input(shape=(4,))
dense1 = Dense(50, activation='relu')(input)
dense2 = Dense(80, activation='relu')(input)
dense3 = Dense(30, activation='relu')(input)
x = concatenate([dense1, dense2, dense3])
output = Dense(3, activation='softmax')(x)
model = Model(inputs=input, outputs=output)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 50)        │        250 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 80)        │        400 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 30)        │        150 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 160)       │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_1[0][0],    │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 3)         │        483 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,283 (5.01 KB)

 Trainable params: 1,283 (5.01 KB)

 Non-trainable params: 0 (0.00 B)

In [2]:
#iris dataset load
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [3]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(X_train, y_train, epochs=300, verbose=1)
history

Epoch 1/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.2480 - loss: 1.4153
Epoch 2/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3330 - loss: 1.2384
Epoch 3/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2861 - loss: 1.1591
Epoch 4/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3337 - loss: 1.0717
Epoch 5/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3732 - loss: 0.9907
Epoch 6/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6413 - loss: 0.9233 
Epoch 7/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7062 - loss: 0.8633 
Epoch 8/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7392 - loss: 0.8007 
Epoch 9/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7922 - loss: 0.7682
Epoch 10/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8110 - loss: 0.7374
Epoch 11/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8235 - loss: 0.7072 
Epoch 12/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7280 

In [5]:
loss, acc = model.evaluate(X_test, y_test)
loss, acc

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.9748 - loss: 0.0831


(0.0768456980586052, 0.9777777791023254)

예측

In [8]:
import numpy as np
pred_proba = model.predict(X_test[:5])
pred_class = np.argmax(pred_proba, axis=1)
pred_class

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


array([0, 1, 1, 0, 2])

In [13]:
from sklearn.metrics import classification_report
y_pred = pred_proba = model.predict(X_test).argmax(axis=1)
print(classification_report(y_test, y_pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      0.94      0.97        18
           2       0.93      1.00      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



모델의 저장

In [14]:
model.save("iris_model.keras")

모델 로드(load)

In [15]:
from tensorflow.keras.models import load_model
model = load_model("iris_model.keras")

In [17]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 50)        │        250 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 80)        │        400 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 30)        │        150 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 160)       │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_1[0][0],    │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 3)         │        483 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,851 (15.05 KB)

 Trainable params: 1,283 (5.01 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,568 (10.04 KB)

In [18]:
json_str = model.to_json()
json_str

'{"module": "keras.src.models.functional", "class_name": "Functional", "config": {"name": "functional", "trainable": true, "layers": [{"module": "keras.layers", "class_name": "InputLayer", "config": {"batch_shape": [null, 4], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_layer"}, "registered_name": null, "name": "input_layer", "inbound_nodes": []}, {"module": "keras.layers", "class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": {"module": "keras", "class_name": "DTypePolicy", "config": {"name": "float32"}, "registered_name": null}, "units": 50, "activation": "relu", "use_bias": true, "kernel_initializer": {"module": "keras.initializers", "class_name": "GlorotUniform", "config": {"seed": null}, "registered_name": null}, "bias_initializer": {"module": "keras.initializers", "class_name": "Zeros", "config": {}, "registered_name": null}, "kernel_regularizer": null, "bias_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, 

In [19]:
import json
json_dic = json.loads(json_str)
with open('iris_model.json', 'w') as f:
  json.dump(json_dic, f, indent=4)

In [16]:
import numpy as np
pred = np.argmax(model.predict(X_test), axis=1)
np.mean(np.equal(y_test, pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


np.float64(0.9777777777777777)